# Adult Income Exercise

For this exercise we will use a dataset that contains some features about people and we need to predict <b>if each person is payed more than 50.000 $ annualy or not</b>.

- You need to:
    - Explore the dataset
    - Give some basic information about the data
    - Preprocess the data (missing values, imputation...)
    - Test various Machine Learning algorithms
    - Evaluate the performance of these algorithms over the test data in terms of Accuracy, Precision, Recall, F1, plot the confusion matrix...
    - Chose one of these algorithms to perfom the task prediction and explain why (in comments or in a markdown cell)



In [9]:
import pandas as pd





In [10]:
#Your code hereimport pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn import metrics
import bisect

import warnings
warnings.filterwarnings('ignore')
# Read in the data
df_train = pd.read_csv('data/train.csv')
df_train.replace('', pd.NA, inplace=True)
df_train = df_train.dropna()
X_train = df_train.drop(columns=["Survived"], axis=1)
X_test = pd.read_csv('data/test.csv')
y_train = df_train[["Survived"]]



numeric_columns = X_train.dtypes[((df_train.dtypes=="float64")|(X_train.dtypes=="int64"))].index.values.tolist()


categorical_col = X_train.dtypes[df_train.dtypes=="object"].index.values.tolist()



mmc = StandardScaler()

scaled_columns = [x for x in numeric_columns if x != 'Survived']
print(numeric_columns)

X_train[scaled_columns] = mmc.fit_transform(X_train[scaled_columns])
X_test[scaled_columns] = mmc.transform(X_test[scaled_columns])


ohe = OneHotEncoder(drop = None, handle_unknown='ignore', sparse_output=False)

ohe_cols = ["Name", "Sex", "Ticket"]

X_train_encoded = pd.DataFrame(ohe.fit_transform(X_train[ohe_cols]), index=X_train.index)
X_test_encoded = pd.DataFrame(ohe.transform(X_test[ohe_cols]), index=X_test.index)

X_train_encoded.columns = ohe.get_feature_names_out(ohe_cols)
X_test_encoded.columns = ohe.get_feature_names_out(ohe_cols)

X_train.drop(ohe_cols ,axis=1, inplace=True)
X_test.drop(ohe_cols ,axis=1, inplace=True)

X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded ], axis=1)

## Gradient Boosting

# Create the random grid
random_grid = { # Number of trees in gradient boost
               'n_estimators': [int(x) for x in np.linspace(start = 50, stop = 400, num = 8)],
                # Number of features to consider at every split
               'max_features': ['auto', 'sqrt', 'log2'],
                # Maximum number of levels in tree
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)] + [None],
                # Minimum number of samples required to split a node
               'min_samples_split': [2, 5, 10],
                # Minimum number of samples required at each leaf node
               'min_samples_leaf': [1, 2, 4],
               'learning_rate': [0.1, 1],
               'subsample': [0.5, 0.7, 0.9]}

print(random_grid)
gb = GradientBoostingClassifier(random_state=0)
gb_search = RandomizedSearchCV(estimator = gb, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=1, random_state=0)
gb_search = gb_search.fit(X_train, y_train)

y_train_pred = gb.predict(X_train)
y_test_pred = gb.predict(X_test)


result = pd.concat([X_test["PassengerId"], pd.DataFrame(y_test_pred)], axis=1, ignore_index=True)

result.columns=["PassengerId", "Survived"]

result.to_csv("result.csv", index=False)




['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
{'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'learning_rate': [0.1, 1], 'subsample': [0.5, 0.7, 0.9]}
Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/home/deimos/Documents/Machine_Learning_ECE/TD3 Exos-20241017/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/deimos/Documents/Machine_Learning_ECE/TD3 Exos-20241017/.venv/lib/python3.12/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/deimos/Documents/Machine_Learning_ECE/TD3 Exos-20241017/.venv/lib/python3.12/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/deimos/Documents/Machine_Learning_ECE/TD3 

### Chose one of the algorithms and explain why

Your answer here:

Après analyse de l'accuracy et des valeurs f1 score, on en conclut que l'algorithme le plus performant est le gradient boosting.